In [264]:
reset -f

In [265]:
import pandas as pd
import numpy as np
import ast
import re

# Classifiers importation
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import neural_network
from sklearn.naive_bayes import BernoulliNB

# Others
import seaborn as sns
from sklearn import linear_model, metrics, model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

In [266]:
# Intervals resposta per y_fc y_rt

class segmenta():
    
    def __init__(self):
        self.intervals = {}
    
    def transform(self,y):
        k, z = 0, 0
        for a, b in [[15,5],
                     [25,10],
                     [50,25],
                     [100,50],
                     [1000,900],
                     [10000,4500],
                     [50000,10000]]: 
            for i in np.arange(z,a,b):
                self.intervals[k] = [i,i+b-1]
                k+=1
            z = a
        self.intervals[k] = [50000,99999999]

        C = np.zeros(len(y))
        for j in self.intervals:
            c = self.intervals[j]
            idx = np.where(np.logical_and(y>=c[0],y<=c[1]))[0]
            C[idx] = j

        return C
seg = segmenta()

Primer accedim al csv de clean_newauthors per tal d'aplicar un model que separi en 3 classes per FC (0-15-50-inf) i 3 classes per RT (0-5-25-inf).


Reproduim aqui el que fa el fitxer read_clean_newauthors

In [267]:
# Carreguem les dades d'autor de la persona que posa el tweet.
# Informacio necessaria:
# j_user 	RT_l10 	sd_RT 	FC_l10 	sd_FC 
file = pd.read_csv('author_prova.csv')
file.drop('Unnamed: 0',axis=1,inplace=True)

#en aquest cas, ens estem quedant amb la primera fila del fitxer 'author.csv'

In [268]:
# Columnes que necessitem
cols_num = ['followers_count','friends_count','listed_count','favourites_count','statuses_count']

# Creem les columnes definides
for col in cols_num:
    file[col] = 0
file['j_user'][0] = ast.literal_eval(file['j_user'][0])
for col in cols_num:
    file[col][0] = int(file['j_user'][0][col])
    
file.drop('j_user',axis=1,inplace=True)
# Ara en principi ja hauria de tenir el format de clean_newauthors.csv
# S'haura d'aplicar el model per predir la classe de FC i la classe de RT

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


#### Predim classe per FC

La variable X utilitzada és:

In [269]:
# Asseguremnos que les variables d'entrada son aquestes i les tenim
X = file[['RT_l10', 'sd_RT', 'FC_l10', 'sd_FC', 'friends_count', 'followers_count']]
from sklearn.externals import joblib
def load_model(filename):
    return joblib.load(filename)
model_class_fc = load_model('trained_groupsFC_Log.sav')
class_fc = model_class_fc.predict(X).reshape(-1,1)

i la classe de FC a la que pertany s'obté utilizant 'trained_groupsFC_Log.sav' , i pren valors 0, 1 o 2.

In [270]:
if (class_fc == 0): 
    y_fc = X['FC_l10']
    y_fc = pd.DataFrame(y_fc)
    y_fc = seg.transform(y_fc)
    y_fc = seg.intervals[y_fc[0]]
if (class_fc !=0):
    # Carregar les dades del tweet: filename = './tweet.csv'
    # Arreglem per aplicar el model
    file = pd.read_csv("tweet_prova.csv")
    file = file[['Tweet']]
    
    mu = X[['FC_l10']]# mitjana
    sd = X[['sd_FC']]# desviacio
    if(class_fc == 1):
        vect_FC_C1 = load_model('vect_FC_C1')
        x = vect_FC_C1.transform(file).todense()
        FC_C1_tweet = load_model('FC_C1_tweet')
        y_fc = FC_C1_tweet.predict(x)
    elif(class_fc ==2):
        vect_FC_C2 = load_model('vect_FC_C2')
        x = vect_FC_C2.transform(file).todense()
        FC_C2_tweet = load_model('FC_C2_tweet')
        y_fc = FC_C2_tweet.predict(x)
        
    k = seg.transform(pd.DataFrame([mu]))[0]

    if (y_fc > mu + sd*1.96) or (y_fc < mu - sd*1.96):
        y_fc = seg.intervals[k]
    else: 
        y_fc = seg.intervals[pred]


#### Predim classe per RT

La variable X utilitzada és:

In [271]:
X = file[['RT_l10', 'sd_RT', 'FC_l10', 'sd_FC', 'friends_count', 'followers_count', 'listed_count', 'favourites_count', 'statuses_count']]
# Carreguem el model que ho prediu: 'groupsRT_Log'
model_class_rt = load_model('trained_groupsRT_Log.sav')
class_rt = model_class_rt.predict(X).reshape(-1,1)

i la classe de FC a la que pertany s'obté utilizant 'trained_groupsRT_Log.sav' , i pren valors 0, 1 o 2.

In [272]:
if (class_rt == 0):
    y_rt = X['RT_l10']
    y_rt = pd.DataFrame(y_rt)
    y_rt = seg.transform(y_rt)
    y_rt = seg.intervals[y_rt[0]]
if (class_rt !=0):
    # Carregar les dades del tweet: filename = './tweet.csv'
    # Arreglem per aplicar el model
    # JA esta carregat
    #file = pd.read_csv("tweet.csv")
    #file = file[['Tweet']]
    
    mu = X[['RT_l10']]# mitjana
    sd = X[['sd_RT']]# desviacio
    if(class_rt == 1):
        vect_RT_C1 = load_model('vect_RT_C1')
        x = vect_RT_C1.transform(file).todense()
        RT_C1_tweet = load_model('RT_C1_tweet')
        y_rt = FC_C1_tweet.predict(x)
    elif(class_rt ==2):
        vect_RT_C2 = load_model('vect_RT_C2')
        x = vect_RT_C2.transform(file).todense()
        FC_RT_tweet = load_model('RT_C2_tweet')
        y_rt = FC_RT_tweet.predict(x)
        
    k = seg.transform(pd.DataFrame([mu]))[0]

    if (y_rt > mu + sd*1.96) or (y_rt < mu - sd*1.96):
        y_rt = seg.intervals[k]
    else: 
        y_rt = seg.intervals[pred]
